### Problem formulation

some text here
<br/><br/><br/>

In [16]:
import findspark
findspark.init()

### Import all needed libs

In [17]:
# basically spark
import pyspark
import operator
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.window import Window
from pyspark.sql.types import ArrayType, FloatType, StringType, IntegerType
from pyspark.sql.functions import udf, row_number,column

# processing
import re
from datetime import datetime

# text preprocessing
import nltk
from nltk.corpus import stopwords
from pyspark.ml.feature import CountVectorizer,StopWordsRemover, HashingTF, IDF, Tokenizer

#staff for LDA
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vector as oldVector, Vectors as oldVectors
from pyspark.ml.linalg import Vector as newVector, Vectors as newVectors

#for debug purpose only
import time

In [18]:
sc = SparkContext()
spark = SparkSession(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-3-73ad000ecbc7>:1 

### Global variables definition

**User-specific variables**  
Please feel free to tweak those variables as you wish. For example, you can set number of last hours to get hottest topics.

In [4]:
geo = 'US-NY' #US for USA

number_of_hours_to_get_topics = 2

# Set window time for interesting
frame_start_datetime = "Sun Jun 27 00:00:00 +0000 2019"
frame_finish_datetime = "Sun Jun 30 23:00:00 +0000 2019"

**Technical variables**  
Those variables are needed to connect to db and other technical stuff.

In [5]:
num_of_topics_LDA = 10
max_iterations_LDA = 100
nomber_of_words_to_for_topic = 15  # number of words per topic

historical_tweets_data = './get-tweets-by-geolocation/training_tweets.csv'
real_time_tweets_table = "usa_training_tweets_02_07.training_tweets_collection"

### Handy functions

In [6]:
def str_tweet_to_datetime(frame_datetime):
    ts = datetime.strptime(frame_datetime,'%a %b %d %H:%M:%S %z %Y')
    return ts

def datetime_to_tweet_str(frame_datetime):
    #print(type(frame_datetime))
    ts = datetime.strftime(frame_datetime, '%a %b %d %H:%M:%S %z %Y')
    return ts

def tweet2google_timeframe(frame_start_datetime, frame_finish_datetime):
    start_date = str_tweet_to_datetime(frame_start_datetime)
    end_date = str_tweet_to_datetime(frame_finish_datetime)
    tim
    
def get_google_trends_by_geo(geo):
    if geo == 'US':
        return google_trends_search_topics_us, google_trends_search_queries_us
    elif geo == 'US-NY':
        return google_trends_search_topics_us_ny, google_trends_search_queries_us_ny
    
    return None, None

In [7]:
#TODO: move this function to utils
def str_rising_to_float(str):
    if str is None:
        return 0.0
    if str == '':
        return 0.0
    if str == 'Breakout':
        return 0.0
    
    str_value = str.split('%')[0]
    if '+' in str_value:
        str_value = str_value.split('+')[1]
        
    if ',' in str_value:
        str_value = str_value.replace(',', '.')
        value = 1000* float(str_value)
        return value
    return float(str_value)

In [8]:
#TODO: move this function to utils
def unique_google_trends_by_time_frame(df):
    data = df.collect()
    rising_dict = {}
    top_dict = {}
    
    geo = data[0]['geo']
    columns = df.columns

    for i in range(0, len(data)):
        rising_val = data[i][columns[1]]
        top_value = data[i][columns[2]]
        
        if rising_val in rising_dict:
            rising_dict[rising_val][0] += str_rising_to_float(data[i][columns[3]])
            rising_dict[rising_val][1] += 1
        else:
            rising_dict[rising_val] = [str_rising_to_float(data[i][columns[3]]), 1]
            
        if top_value in top_dict:
            top_dict[top_value][0] += float(data[i][columns[4]])
            top_dict[top_value][1] += 1
        else:
            top_dict[top_value] = [float(data[i][columns[4]]), 1]
    
    
    for key in top_dict:
        top_dict[key] = round(top_dict[key][0] / top_dict[key][1])
        
    for key in rising_dict:
        rising_dict[key] = round(rising_dict[key][0] / rising_dict[key][1])
    
    top_dict = sorted(top_dict.items(), key=operator.itemgetter(1), reverse=True)
    rising_dict = sorted(rising_dict.items(), key=operator.itemgetter(1), reverse=True)
    
    
    seq = []
    len_top = len(top_dict)
    len_rising = len(rising_dict)
    length = max(len_top, len_rising)
    
    row = Row(columns[1], columns[2], columns[3], columns[4], columns[5])
    
    for i in range(0, length):
        rising = rising_dict[i][0] if i < len_rising else ''
        rising_val = f"+{rising_dict[i][1]}%" if i < len_rising else None
        
        top = top_dict[i][0] if i < len_top else ''
        top_val = top_dict[i][1] if i < len_top else None
        
        seq.append(row(rising, top, rising_val, top_val, geo))
    
    dframe = spark.createDataFrame(seq)
    return dframe

# Load the data


## Here should be "magic IF" (Yevhen)

In [9]:
# df = spark.read.csv(historical_tweets_data, inferSchema=True, header=True)
df = spark.read.csv('./get-tweets-by-geolocation/data/new_york_training_tweets_15_06.csv', inferSchema=True, header=True)

In [10]:
print(type(df))
print(df.columns)
df.dtypes

<class 'pyspark.sql.dataframe.DataFrame'>
['tweet', 'country_code', 'geo_location', 'bounding_box', 'screen_name', 'favourites_count', 'followers_count', 'statuses_count', 'friends_count', 'listed_count', 'user_described_location', 'created_at', 'utc_offset']


[('tweet', 'string'),
 ('country_code', 'string'),
 ('geo_location', 'string'),
 ('bounding_box', 'string'),
 ('screen_name', 'string'),
 ('favourites_count', 'string'),
 ('followers_count', 'string'),
 ('statuses_count', 'string'),
 ('friends_count', 'string'),
 ('listed_count', 'string'),
 ('user_described_location', 'string'),
 ('created_at', 'string'),
 ('utc_offset', 'string')]

In [ ]:
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType

# Setting an user define function:
# This function converts the string cell into a date:
func =  udf (lambda x: str_tweet_to_datetime(x), DateType())

df = df.withColumn('test', func(col('created_at')))

df.dtypes
df.show()

In [14]:
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType

def nothing(x):
    return x

# Setting an user define function:
# This function converts the string cell into a date:
func =  udf (lambda x: str_tweet_to_datetime(x), DateType())
# func =  udf (lambda x: nothing(x), StringType())

df = df.withColumn('tweet_data', func(col('created_at')))

print(df.dtypes)
# df.show()

df.select("tweet_data").show(10, False)

# df1.printSchema()

[('tweet', 'string'), ('country_code', 'string'), ('geo_location', 'string'), ('bounding_box', 'string'), ('screen_name', 'string'), ('favourites_count', 'string'), ('followers_count', 'string'), ('statuses_count', 'string'), ('friends_count', 'string'), ('listed_count', 'string'), ('user_described_location', 'string'), ('created_at', 'string'), ('utc_offset', 'string'), ('tweet_data', 'date')]


Py4JJavaError: An error occurred while calling o137.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5.0 (TID 6, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 345, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 141, in dump_stream
    for obj in iterator:
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 334, in _batched
    for item in iterator:
  File "<string>", line 1, in <lamb

### Connecting to MongoDB for the real-time data
**Creating Spark connector**

In [3]:
spark = SparkSession.builder.appName("pipeline") \
    .config('spark.mongodb.input.uri', 'mongodb://localhost:27017/'+tweets_table) \
    .config('spark.mongodb.output.uri', 'mongodb://localhost:27017/'+tweets_table) \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.3.1') \
    .config('spark.mongodb.input.partitioner', 'MongoPaginateBySizePartitioner') \
    .getOrCreate()

In [4]:
df = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()

In [5]:
df.createOrReplaceTempView("recent_data")

In [8]:
df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- contributors: null (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- type: string (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- monetizable: boolean (nullable = true)
 |    |    |    |-- dis

In [24]:
recent_data = spark.sql('SELECT text FROM recent_data WHERE lang="en" AND place.country="US" LIMIT 10')

In [25]:
recent_data.show()

+----+
|text|
+----+
+----+



# Here should be Main dataframe already filtered

## Loading Google Trends data

In [20]:
google_trends_search_queries_us = spark.read.csv('data/google-trends/google-trends-search-queries-US.csv', inferSchema=True, header=True)
google_trends_search_topics_us = spark.read.csv('data/google-trends/google-trends-search-topics-US.csv', inferSchema=True, header=True)
google_trends_search_queries_us_ny = spark.read.csv('data/google-trends/google-trends-search-queries-US-NY.csv', inferSchema=True, header=True)
google_trends_search_topics_us_ny = spark.read.csv('data/google-trends/google-trends-search-topics-US-NY.csv', inferSchema=True, header=True)

# Data description (Женя)
In order to collect data in a natural way:
<br>- we registered Twitter Developer account;
<br>- using credentials from Twitter Developer account we run script that collected tweets by the geolocation and saved them in mongodb;
<br>
<br><b>As a result:</b>
<br>- we collected  332548 tweets (10Gb in mongodb, ~100Mb in csv) from New-York geolocation since 30 of May up to 15 of June;
<br>- we collected  6617029 tweets (~1.69Gb in csv) from USA geolocation since 15 of June up to now.

In [1]:
#show Raw data

### Data preprocessing
# description

#### Selecting data from {{ frame_start_datetime }} to {{frame_finish_datetime}}

In [59]:
StopWords = stopwords.words("english")

tokens = df.select('text').rdd \
    .map(lambda x: x[0]) \
    .map( lambda document: re.split(" ", document))          \
    .map( lambda word: [x for x in word if x.isalpha()])           \
    .map( lambda word: [x for x in word if len(x) > 3] )           \
    .map( lambda word: [x for x in word if x not in StopWords])    \


AnalysisException: "cannot resolve '`text`' given input columns: [];;\n'Project ['text]\n+- Relation[] MongoRelation(MongoRDD[14] at RDD at MongoRDD.scala:51,Some(StructType()))\n"

In [60]:
tokens.collect()

NameError: name 'tokens' is not defined

In [2]:
#show preprocessed data

### Topic modeling/Latent Dirichlet allocation(LDA)

In [75]:
# this block can be commented, it's just a mock

text_file = 'data/listings.csv'
df = spark.read.csv(text_file, inferSchema=True, header=True)
df = df.select("id", "name").dropna(subset="name")

print(time.strftime('%m%d%Y %H:%M:%S'))

tokenizer = Tokenizer(inputCol="name", outputCol="tokens")
df = tokenizer.transform(df)
print(time.strftime('%m%d%Y %H:%M:%S'))

07042019 23:23:25
07042019 23:23:25


In [76]:
print(time.strftime('%m%d%Y %H:%M:%S'))

cv = CountVectorizer(inputCol="tokens", outputCol="raw_features", vocabSize=5000, minDF=3.0)
cvmodel = cv.fit(df)

print(time.strftime('%m%d%Y %H:%M:%S'))

07042019 23:23:25
07042019 23:23:26


In [77]:
print(time.strftime('%m%d%Y %H:%M:%S'))
df = cvmodel.transform(df)
print(time.strftime('%m%d%Y %H:%M:%S'))

07042019 23:23:26
07042019 23:23:26


In [78]:
idf = IDF(inputCol="raw_features", outputCol="tf_idf_features", minDocFreq=2)
idfModel = idf.fit(df)

df = idfModel.transform(df)


In [90]:
df.show(10, True)

+---+--------------------+--------------------+--------------------+--------------------+
| id|                name|              tokens|        raw_features|     tf_idf_features|
+---+--------------------+--------------------+--------------------+--------------------+
|  1|            27987182|          [27987182]|        (1778,[],[])|        (1778,[],[])|
|  2|            21686664|          [21686664]|        (1778,[],[])|        (1778,[],[])|
|  3|            21686664|          [21686664]|        (1778,[],[])|        (1778,[],[])|
|  4|            17607060|          [17607060]|        (1778,[],[])|        (1778,[],[])|
|  5|             5630387|           [5630387]|        (1778,[],[])|        (1778,[],[])|
|  6|YAYS Bickersgrach...|[yays, bickersgra...|(1778,[265,694,77...|(1778,[265,694,77...|
|  7|YAYS Bickersgrach...|[yays, bickersgra...|(1778,[265,694,96...|(1778,[265,694,96...|
|  8|Roof Terrace Cozy...|[roof, terrace, c...|(1778,[16,45,62,8...|(1778,[16,45,62,8...|
|  9|Amazi

In [63]:
#df = df.drop("name")
#df.show(10, False)

In [84]:
w = Window().orderBy(column("id"))
df = df.withColumn("id", row_number().over(w))

In [91]:
rs = df.rdd.map(lambda x: (x[0], oldVectors.fromML(x[4])))

In [92]:
rs_df = rs.toDF()
rs_df.show(10, False)

+---+--------------------------------------------------------------------------------------------------------------------------------------------+
|_1 |_2                                                                                                                                          |
+---+--------------------------------------------------------------------------------------------------------------------------------------------+
|1  |(1778,[],[])                                                                                                                                |
|2  |(1778,[],[])                                                                                                                                |
|3  |(1778,[],[])                                                                                                                                |
|4  |(1778,[],[])                                                                                                     

In [93]:
# Run the LDA Topic Modeler
# Note the time before and after is printed in order to find out how much time it takes to process x number of records

print(time.strftime('%m%d%Y %H:%M:%S'))
lda_model = LDA.train(rs_df['_1', '_2'].rdd.map(list), k=num_of_topics_LDA, maxIterations=max_iterations_LDA)
print(time.strftime('%m%d%Y %H:%M:%S'))

07042019 23:27:34
07042019 23:28:31


In [94]:
print(time.strftime('%m%d%Y %H:%M:%S'))
topics = lda_model.topicsMatrix()
vocabArray = cvmodel.vocabulary

topicIndices = sc.parallelize(lda_model.describeTopics(maxTermsPerTopic = wordNumbers))

def topic_render(topic):  # specify vector id of words to actual words
    terms = topic[0]
    prob = topic[1]
    
    result = []
    for i in range(nomber_of_words_to_for_topic):
        term = str(round(prob[i],3))+"  "+vocabArray[terms[i]]
        result.append(term)
    return result
print(time.strftime('%m%d%Y %H:%M:%S'))

07042019 23:29:31
07042019 23:29:32


In [95]:
print(time.strftime('%m%d%Y %H:%M:%S'))
topics_final = topicIndices.map(lambda topic:topic_render(topic)).collect()
print(time.strftime('%m%d%Y %H:%M:%S'))

07042019 23:29:33
07042019 23:29:33


# Topics

In [88]:
# based on the simple vectors(+number of words)

for topic in range(len(topics_final)):
    print ("Topic #" + str(topic+1) + "")
    for term in topics_final[topic]:
        print (term)
    print ('\n')

Topic #1
0.057  for
0.055  apartment
0.045  in
0.036  a
0.036  bedroom
0.031  2
0.027  with
0.025  amsterdam
0.016  canal
0.014  room
0.014  of
0.013  view
0.012  the
0.011  1
0.011  house


Topic #2
0.077  apartment
0.066  in
0.042  amsterdam
0.027  with
0.022  spacious
0.022  and
0.018  near
0.017  &
0.015  the
0.015  city
0.013  garden
0.012  cosy
0.012  room
0.012  cozy
0.012  bright


Topic #3
0.07  apartment
0.056  with
0.048  in
0.028  amsterdam
0.026  spacious
0.019  garden
0.018  and
0.017  the
0.017  &
0.017  house
0.015  near
0.014  city
0.013  modern
0.012  family
0.012  terrace


Topic #4
0.07  apartment
0.062  in
0.039  amsterdam
0.03  with
0.022  spacious
0.022  &
0.02  the
0.019  to
0.017  house
0.016  room
0.016  and
0.016  city
0.015  cosy
0.014  garden
0.013  near


Topic #5
0.066  apartment
0.066  in
0.044  amsterdam
0.028  the
0.027  centre
0.027  city
0.023  to
0.017  with
0.016  spacious
0.016  near
0.016  room
0.014  center
0.014  of
0.013  and
0.013  &


Topic 

In [96]:
#based on the tf-idf

for topic in range(len(topics_final)):
    print ("Topic #" + str(topic+1) + "")
    for term in topics_final[topic]:
        print (term)
    print ('\n')

Topic #1
0.103  house
0.081  family
0.079  beautiful
0.049  on
0.044  at
0.038  houseboat
0.037  amsterdam
0.033  new
0.033  perfect
0.023  two
0.014  north
0.013  warm
0.012  suite
0.012  historic
0.011  ★


Topic #2
0.127  the
0.08  of
0.055  bedroom
0.04  comfortable
0.039  luxury
0.037  heart
0.032  large
0.03  free
0.028  amsterdam
0.028  parking
0.024  3
0.022  in
0.02  1
0.02  double
0.019  stay


Topic #3
0.109  with
0.085  garden
0.067  a
0.062  home
0.061  for
0.051  area
0.039  in
0.037  east
0.036  trendy
0.029  apt.
0.019  romantic
0.016  comfy
0.013  area!
0.012  apartment
0.011  friendly


Topic #4
0.107  city
0.093  centre
0.088  to
0.082  center
0.055  close
0.037  next
0.035  from
0.022  min
0.018  park
0.017  district
0.016  spacious,
0.015  centre!
0.013  15
0.013  10
0.013  stunning


Topic #5
0.096  near
0.068  lovely
0.067  cozy
0.066  cosy
0.056  vondelpark
0.05  nice
0.045  west
0.034  appartement
0.025  amsterdam
0.02  old
0.02  clean
0.02  apartment
0.015  sq

### Hot topics in the USA from [Google trends](https://trends.google.com/trends/explore?geo=US)

In [22]:
start_date = str_tweet_to_datetime(frame_start_datetime)
finish_date = str_tweet_to_datetime(frame_finish_datetime)

In [24]:
google_trends_topics, google_trends_queries = get_google_trends_by_geo(geo) 

Google trends topics

In [25]:
interesing_google_topics = google_trends_topics.filter(
    (google_trends_topics.Date >= start_date) & (google_trends_topics.Date <= finish_date))

In [31]:
interesing_google_topics.select("Date","Search topics - rising", "Search topics - top").show(15, False)

+-------------------+-------------------------------------------------------------+---------------------------------------+
|Date               |Search topics - rising                                       |Search topics - top                    |
+-------------------+-------------------------------------------------------------+---------------------------------------+
|2019-06-27 00:00:00|Software - Software grouping                                 |New York - City in New York            |
|2019-06-27 00:00:00|Academic degree - Degree                                     |New York - US State                    |
|2019-06-27 00:00:00|Loan - Topic                                                 |2019 - Topic                           |
|2019-06-27 00:00:00|Management - Topic                                           |Hotel - Building function              |
|2019-06-27 00:00:00|Bra - Dress                                                  |Weather - Topic                        |
|2019-06

In case when timeframe is more than 1 day, filter correctly this google-trends

In [218]:
interesing_google_topics_unique= unique_google_trends_by_time_frame(interesing_google_topics)
interesing_google_topics_unique.show()

+----------------------+--------------------+------+---+-----+
|Search topics - rising| Search topics - top|Rising|Top|  geo|
+----------------------+--------------------+------+---+-----+
|  Kamala Harris - U...|New York - City i...|+3550%|100|US-NY|
|  Marianne Williams...| New York - US State|+2200%| 90|US-NY|
|  Brooklyn Nets - B...|        2019 - Topic|+1050%| 42|US-NY|
|  United States wom...|     Weather - Topic| +750%| 37|US-NY|
|  Pete Buttigieg - ...|Hotel - Building ...| +650%| 28|US-NY|
|  Joe Biden - Forme...|Google Search - T...| +600%| 24|US-NY|
|  Kevin Durant - Am...|Google - Technolo...| +600%| 24|US-NY|
|  Boston Red Sox - ...|YouTube - Video s...| +400%| 24|US-NY|
|         Radar - Topic|        Film - Topic| +350%| 24|US-NY|
|    Free agent - Topic|Facebook, Inc. - ...| +350%| 23|US-NY|
|  Software - Softwa...|Facebook - Social...| +250%| 23|US-NY|
|  Academic degree -...|       Sales - Topic| +250%| 20|US-NY|
|  FIFA Women's Worl...|United States - C...| +250%| 20

Google trends queries

In [48]:
interesing_google_queries = google_trends_queries.filter(
    (google_trends_queries.Date >= start_date) & (google_trends_queries.Date <= finish_date))

In [49]:
interesing_google_queries.show()

+-------------------+-----------------------+--------------------+-------+---+-----+
|               Date|Search queries - rising|Search queries - top| Rising|Top|  geo|
+-------------------+-----------------------+--------------------+-------+---+-----+
|2019-06-27 00:00:00|         gerrymandering|                 you|+4,250%|100|US-NY|
|2019-06-27 00:00:00|          tulsi gabbard|             weather|+1,850%| 74|US-NY|
|2019-06-27 00:00:00|            cory booker|              google|+1,450%| 73|US-NY|
|2019-06-27 00:00:00|          julian castro|            facebook|+1,200%| 62|US-NY|
|2019-06-27 00:00:00|   india vs west ind...|                news|  +750%| 48|US-NY|
|2019-06-27 00:00:00|           john delaney|              amazon|  +450%| 47|US-NY|
|2019-06-27 00:00:00|       elizabeth warren|             youtube|  +400%| 45|US-NY|
|2019-06-27 00:00:00|              ind vs wi|              debate|  +250%| 36|US-NY|
|2019-06-27 00:00:00|              powerball|           world cup

In [217]:
interesing_google_queries_unique= unique_google_trends_by_time_frame(interesing_google_queries)
interesing_google_queries_unique.show()

+-----------------------+--------------------+------+---+-----+
|Search queries - rising|Search queries - top|Rising|Top|  geo|
+-----------------------+--------------------+------+---+-----+
|                     yy|                 you|+4950%|100|US-NY|
|         gerrymandering|             weather|+4250%| 94|US-NY|
|          shay mitchell|               pride|+3700%| 62|US-NY|
|        darren collison|              google|+2950%| 60|US-NY|
|    marianne williamson|            facebook|+2750%| 60|US-NY|
|          kamala harris|             youtube|+2400%| 48|US-NY|
|         deandre jordan|                news|+2400%| 45|US-NY|
|          tulsi gabbard|              amazon|+1850%| 45|US-NY|
|            cory booker|           world cup|+1450%| 38|US-NY|
|          julian castro|              debate|+1200%| 36|US-NY|
|       india vs england|             yankees|+1050%| 34|US-NY|
|   argentina vs vene...|        pride parade| +900%| 30|US-NY|
|             usa france|               

#### Hot topics - google trends (directly) (probably this will be removed)

In [118]:
!ls

data			   README.md
get-google-trends-data	   realtime_twitter_topic_modeling_pyspark.ipynb
get-tweets-by-geolocation  sample_lda_data.txt
LICENSE			   spark_LDA.ipynb
python-data-preprocessing  spark_LDA_v2.ipynb
python_LDA.ipynb	   spark-warehouse


In [128]:
from get_google_trends_data.pytrends.pytrends.request import TrendReq
pytrend = TrendReq()

timeframe = "2019-06-5 2019-06-20"
pytrend.build_payload(kw_list=[' '], geo='US', timeframe=timeframe)
topics_df = pytrend.related_top_topics_by_top()

In [129]:
topics_df

,Search topics,Top
0,2019 - Topic,100
1,Weather - Topic,85
2,Facebook - Social networking service,73
3,"Facebook, Inc. - Social network company",73
4,Google Search - Topic,69
5,Google - Technology company,67
6,YouTube - Video sharing company,64
7,Film - Topic,60
8,Car - Transportation mode,47
9,Texas - US State,46


### Conclusion